In [20]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(11,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='tanh'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

/opt/miniconda3/envs/kaggleEnv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [4]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,337 (9.13 KB)

 Trainable params: 2,337 (9.13 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
from sklearn.preprocessing import LabelEncoder, StandardScaler,OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer

encoder = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'),[4, 2, 9]),
        ('label', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), 
         [5]),  # Adjusted indices after dropping 'id'
        ('scaler', StandardScaler(), 
         [0, 1, 3, 6, 7, 8, 10])  # Adjusted indices after dropping 'id'
    ], remainder='passthrough'
)

In [22]:
df = pd.read_csv('../dataset/train.csv')

In [23]:
X = df.drop(columns=['id', 'loan_status'])
y = df['loan_status']

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [25]:
encoder.fit(X_train)
X_train = encoder.transform(X_train)
X_test = encoder.transform(X_test)
X_val = encoder.transform(X_val)

In [26]:
import tensorflow as tf

# Define the model
model = tf.keras.Sequential([
    # First hidden layer
    tf.keras.layers.Dense(64, activation='relu', input_shape=(20,)),  # Increased neurons
    tf.keras.layers.BatchNormalization(),  # Batch normalization for stable learning
    tf.keras.layers.Dropout(0.3),  # Dropout to prevent overfitting
    
    # Second hidden layer
    tf.keras.layers.Dense(128, activation='relu'),  # Increased neurons
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    
    # Third hidden layer
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    
    # Fourth hidden layer
    tf.keras.layers.Dense(32, activation='relu'),
    
    # Output layer with sigmoid activation for binary classification
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),  # Lower learning rate for fine-tuning
              loss='binary_crossentropy',  # Binary classification
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

/opt/miniconda3/envs/kaggleEnv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
# history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))

In [29]:
# # plot history
# import matplotlib.pyplot as plt
# plt.plot(history.history['accuracy'], label='train')
# plt.plot(history.history['val_accuracy'], label='test')
# plt.legend()
# plt.show()

In [72]:
# draw model boundaries
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("accuracy",accuracy_score(y_test, y_pred))


367/367 ━━━━━━━━━━━━━━━━━━━━ 0s 364us/step
[[9992   95]
 [ 518 1124]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     10087
           1       0.92      0.68      0.79      1642

    accuracy                           0.95     11729
   macro avg       0.94      0.84      0.88     11729
weighted avg       0.95      0.95      0.94     11729

accuracy 0.947736379913036


In [30]:
import keras_tuner as kt

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(hp.Int('units1', min_value=32, max_value=512, step=32), activation='relu', input_shape=(20,)))
    model.add(tf.keras.layers.Dropout(hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(tf.keras.layers.Dense(hp.Int('units2', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(tf.keras.layers.Dropout(hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    
    return model

# Define tuner and run search
tuner = kt.RandomSearch(build_model, 
                        objective='val_accuracy', 
                        max_trials=10, 
                        executions_per_trial=3, 
                        directory='my_dir', 
                        project_name='loan_model')

tuner.search(X_train, y_train, epochs=30, validation_data=(X_val, y_val))

Trial 10 Complete [00h 00m 49s]
val_accuracy: 0.9492753744125366

Best val_accuracy So Far: 0.9498437245686849
Total elapsed time: 04h 18m 39s
